In [14]:
import os, sys
cwd = os.getcwd()
project_path = cwd[:cwd.find('pygents')+7]
if project_path not in sys.path: sys.path.append(project_path)
os.chdir(project_path) 

#from importlib import reload  # Python 3.4+

import pickle
import pandas as pd

#force reimport
if 'pygents.util' in sys.modules:
    del sys.modules['pygents.util']
if 'pygents.text' in sys.modules:
    del sys.modules['pygents.text']
if 'pygents.plot' in sys.modules:
    del sys.modules['pygents.plot']
if 'pygents.token' in sys.modules:
    del sys.modules['pygents.token']
if 'pygents.token_plot' in sys.modules:
    del sys.modules['pygents.token_plot']


from pygents.token import *
from pygents.text import *
from pygents.util import *
from pygents.plot import plot_bars, plot_dict, matrix_plot
from pygents.token_plot import *

In [15]:
path = '../../nlp/corpora/Chinese/'
test_df = pd.read_csv(os.path.join(path,'magicdata/zh_en_ru_100/CORPUS_ZH_EN_RU.txt'),delimiter='\t')
test_texts = list(test_df['zh'])
print(len(test_texts))
test_df[['zh']]

100


,zh
0,然后医疗保险呢？就是我们家，不论是大人啊还是小孩都会去买一个保险
1,当他们买了保险的，按照保险合同的话，是要赔三十万的
2,需要意识到买了一个保险的重要性
3,其实这种现象是真的很普遍，因为比如说你买一个人身意外险你那个你买的越多你那个保额就越多
4,这代父母真的很有必要去买一个保险
...,...
95,蚂蚁保险不止有车险，我看到上面也有各种。
96,但是买房子呢，除了一笔首款付下去之后，每个月的贷款相当于租金
97,这种的投资性的理财，对于个人的财产来说，是具有一定风险
98,如果说你的投资方向正确，那么它是远比存款这一方面要具有更大的优势


In [16]:
for text in test_texts:
    print(text)

然后医疗保险呢？就是我们家，不论是大人啊还是小孩都会去买一个保险
当他们买了保险的，按照保险合同的话，是要赔三十万的
需要意识到买了一个保险的重要性
其实这种现象是真的很普遍，因为比如说你买一个人身意外险你那个你买的越多你那个保额就越多
这代父母真的很有必要去买一个保险
嗯，对现在就是说很提倡买保险
车一定要全款买，房子可以贷款买
可以去买保险，保险当然分为很多类
医疗保险还是很重要的
就是保险公司来出这一部分钱
相互宝，我不知道你有没有了解过，它是支付宝里面的一种保险
买房子其实也就是一个投资
你有没有去了解过平安保险公司的培训啊
如果要是存银行的话，利息利滚利是多少
这个保险在西方国家已经很成熟了
保险行业很多啊，你看它覆盖的面积特别广
但是，其实也是在鼓励你买商业保险和人寿保险
卖保险的都是靠的人脉
呃，你买的社保也是保险呀，然后你的车险也是保险呀
但是银行的钱，你如果是拿出来的话，属于是一种贷款
现在好像转向那个负利率
借银行的钱，我真的没试过
他是没有利息的，所以他就把它放出去，以后再收回来
如果是你征信不好的话，他也不会给钱你用
是他是贷款买的还是全款
嗯，他们就是就是有时候办一张银行卡的时候，程序特别多
各种银行，还有五大银行
国债券也算是证券的一种吗
其实做股票的话嗯相它是算是风险系数比较高的
嗯收益不会太高就跟银行利率是差不多的
然后就用来炒房用啊，等它升值了之后卖一套房，然后还清贷款，可能自己还赚了那个几十万
入投资入门的话，第一步就是说你要掌握基础知识
汇率的标价方式还有他的直接标价法和间接标价法
它始终是一种投资的行为，投资的模式，投资的方式
如果有闲钱呢买一套公寓来投资呢？
变成资金，投资其他的方面
有了贷款就有了压力，也就有了挣钱的动力，这个我也认同
方不方便开通网上银行啊
那你上次投资的钱从哪来的
银行的政策的改变让他们不能贷款了
保险公司呢它也有理财产品的
好像我们那个保险公司是两点八的利息保底
所以很多人对保险很反感
他们宁愿把那个钱去存银行吃利息
提前五年理财真好，因为理财本身就是一个很轻松的事情
就是你把你的生活费支出的一小部分作一个储蓄
债券基金里这种基金的话，它是收益是比较稳定的，就跟存定期一样，但是比定期的钱要多
它理论上可以开多少张这种承兑汇票
就是说我们每个人都应该有自己的理财方式，不管你是你理什么财不管你有多少钱
是

In [20]:
# https://github.com/fxsjy/jieba
zh_jieba_tokenizer = JiebaTokenizer()

# http://www.chineselexicaldatabase.com/download.php - used below
zh_lex_tokenizer = LexiconIndexedTokenizer(lexicon = list(pd.read_csv(os.path.join(path,'lexicon/chineselexicaldatabase2.1.txt'))['Word']))
print(zh_lex_tokenizer.count_params())

# https://www.plecoforums.com/threads/word-frequency-list-based-on-a-15-billion-character-corpus-bcc-blcu-chinese-corpus.5859/
bcc_lex = list(pd.read_csv(os.path.join(path,'lexicon/bcc_global_wordfreq.release_UTF-8.txt'),sep='\t').to_records(index=False))
zh_bcc0_tokenizer = LexiconIndexedTokenizer(lexicon=bcc_lex,sortmode=0)
zh_bcc1_tokenizer = LexiconIndexedTokenizer(lexicon=bcc_lex,sortmode=1)
zh_bcc2_tokenizer = LexiconIndexedTokenizer(lexicon=bcc_lex,sortmode=2)
print(zh_bcc0_tokenizer.count_params())

# http://crr.ugent.be/programs-data/subtitle-frequencies/subtlex-ch
sub_lex = list(pd.read_csv(os.path.join(path,'lexicon/SUBTLEX-CH-WF.txt'),sep='\t')[['Word','WCount']].to_records(index=False))
zh_sub0_tokenizer = LexiconIndexedTokenizer(lexicon=sub_lex,sortmode=0)
zh_sub1_tokenizer = LexiconIndexedTokenizer(lexicon=sub_lex,sortmode=1)
zh_sub2_tokenizer = LexiconIndexedTokenizer(lexicon=sub_lex,sortmode=2)
print(zh_sub0_tokenizer.count_params())



48644
1048575
99121


In [22]:
#http://www.chineselexicaldatabase.com/download.php
print(evaluate_tokenizer_f1(test_texts,zh_jieba_tokenizer,zh_lex_tokenizer,debug=False))
#ttps://www.plecoforums.com/threads/word-frequency-list-based-on-a-15-billion-character-corpus-bcc-blcu-chinese-corpus.5859/
print(evaluate_tokenizer_f1(test_texts,zh_jieba_tokenizer,zh_bcc0_tokenizer,debug=False))#sort by len
print(evaluate_tokenizer_f1(test_texts,zh_jieba_tokenizer,zh_bcc1_tokenizer,debug=False))#sort by freq
print(evaluate_tokenizer_f1(test_texts,zh_jieba_tokenizer,zh_bcc2_tokenizer,debug=False))#sort by len and freq
print(evaluate_tokenizer_f1(test_texts,zh_jieba_tokenizer,zh_sub0_tokenizer,debug=False))#sort by len
print(evaluate_tokenizer_f1(test_texts,zh_jieba_tokenizer,zh_sub1_tokenizer,debug=False))#sort by freq
print(evaluate_tokenizer_f1(test_texts,zh_jieba_tokenizer,zh_sub2_tokenizer,debug=False))#sort by len and freq


0.82
0.79
0.48
0.79
0.82
0.49
0.78


In [26]:
print(len(bcc_lex))
filter_thresholds = [0.000001,0.00001,0.0001,0.001,0.01]
for t in filter_thresholds:
    lex = listofpairs_compress_with_loss(bcc_lex,t)
    print(t,len(lex))
    zh_bcc0_tokenizer = LexiconIndexedTokenizer(lexicon=lex,sortmode=0)
    zh_bcc1_tokenizer = LexiconIndexedTokenizer(lexicon=lex,sortmode=1)
    zh_bcc2_tokenizer = LexiconIndexedTokenizer(lexicon=lex,sortmode=2)
    print(evaluate_tokenizer_f1(test_texts,zh_jieba_tokenizer,zh_bcc0_tokenizer,debug=False))#sort by len
    print(evaluate_tokenizer_f1(test_texts,zh_jieba_tokenizer,zh_bcc1_tokenizer,debug=False))#sort by freq
    print(evaluate_tokenizer_f1(test_texts,zh_jieba_tokenizer,zh_bcc2_tokenizer,debug=False))#sort by len and freq
    print()


1048575
1e-06 141441
0.79
0.48
0.79

1e-05 52857
0.79
0.48
0.79

0.0001 13983
0.76
0.48
0.76

0.001 2499
0.66
0.47
0.66

0.01 185
0.5
0.44
0.5



In [27]:
print(len(sub_lex))
filter_thresholds = [0.000001,0.00001,0.0001,0.001,0.01]
for t in filter_thresholds:
    lex = listofpairs_compress_with_loss(sub_lex,t)
    print(t,len(lex))
    zh_sub0_tokenizer = LexiconIndexedTokenizer(lexicon=lex,sortmode=0)
    zh_sub1_tokenizer = LexiconIndexedTokenizer(lexicon=lex,sortmode=1)
    zh_sub2_tokenizer = LexiconIndexedTokenizer(lexicon=lex,sortmode=2)
    print(evaluate_tokenizer_f1(test_texts,zh_jieba_tokenizer,zh_sub0_tokenizer,debug=False))#sort by len
    print(evaluate_tokenizer_f1(test_texts,zh_jieba_tokenizer,zh_sub1_tokenizer,debug=False))#sort by freq
    print(evaluate_tokenizer_f1(test_texts,zh_jieba_tokenizer,zh_sub2_tokenizer,debug=False))#sort by len and freq
    print()

99121
1e-06 69005
0.83
0.49
0.78

1e-05 29174
0.82
0.49
0.77

0.0001 8343
0.74
0.49
0.72

0.001 1621
0.63
0.49
0.63

0.01 233
0.55
0.46
0.55

